# Tokenizer

In [1]:
import json

In [3]:
data = []
with open('dump_small_clean.jsonln', 'r', encoding="utf8") as file:
    for line in file:
        data.append(json.loads(line))

In [4]:
 import nltk

In [5]:
from nltk import word_tokenize

In [6]:
texto = data[0]['body']

In [7]:
tokens = word_tokenize(texto)

In [8]:
tokens

['{',
 '{',
 'Multitag',
 '{',
 '{',
 'Info/Nome',
 '}',
 '}',
 'Alexandre',
 'é',
 'um',
 'prenome',
 'popular',
 'da',
 'língua',
 'portuguesa',
 '.',
 'É',
 'cognato',
 'ao',
 'nome',
 'Alexander',
 ',',
 'da',
 'língua',
 'inglesa',
 '.',
 'Em',
 'países',
 'lusófonos',
 ',',
 'pessoas',
 'chamadas',
 'Alexandre',
 'são',
 'normalmente',
 'apelidadas',
 'de',
 'Alex',
 '.',
 'Origem',
 'O',
 'nome',
 'deriva',
 'do',
 'latim',
 'Alexander',
 ',',
 'romanização',
 'do',
 'nome',
 'grego',
 'Αλέξανδρος',
 'Aléksandros',
 '.',
 'Etimologicamente',
 ',',
 'o',
 'nome',
 'seria',
 'um',
 'composto',
 'do',
 'verbo',
 'ἀλέξειν',
 'aléksein',
 ',',
 'defender',
 ',',
 'e',
 'o',
 'substantivo',
 'ἀνδρός',
 'andrós',
 ',',
 'genitivo',
 'de',
 'ἀνήρ',
 'anēr',
 ',',
 'homem',
 '.',
 'Assim',
 ',',
 'pode',
 'ser',
 'traduzido',
 'como',
 'protetor',
 'da',
 'Humanidade',
 '.',
 '{',
 '{',
 'carece',
 'de',
 'fontes',
 'O',
 'termo',
 'seria',
 'ou',
 'um',
 'tipo',
 'raro',
 'de',
 'compos

In [9]:
import re


def minusculas(tokens):
    return [token.lower() for token in tokens]

# def remove_digitos(tokens):
#     return [token for token in tokens if re.fullmatch('[^\d]*', token)]

def pega_palavras(tokens):
    return [token for token in tokens if re.fullmatch('\w+', token)]
    
def limpa_tokens(tokens):
    tokens = minusculas(tokens)
#     tokens = remove_digitos(tokens)
    tokens = pega_palavras(tokens)    
    return tokens

In [10]:
from tqdm import tqdm

all_words = []
for item in tqdm(data):
    texto = item['body']
    tokens = word_tokenize(texto)
    tokens = limpa_tokens(tokens)
    all_words += tokens

100%|███████████████████████████████████████████████████████████████████████████| 11225/11225 [00:30<00:00, 364.55it/s]


In [11]:
len(all_words)

2785927

In [12]:
all_words_clean = limpa_tokens(all_words)

In [13]:
len(all_words_clean)

2785927

In [14]:
def stopwords(word_list):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    return [i for i in word_list if i not in stopwords]
stopwords(all_words_clean)

['multitag',
 'alexandre',
 'prenome',
 'popular',
 'língua',
 'portuguesa',
 'cognato',
 'nome',
 'alexander',
 'língua',
 'inglesa',
 'países',
 'lusófonos',
 'pessoas',
 'chamadas',
 'alexandre',
 'normalmente',
 'apelidadas',
 'alex',
 'origem',
 'nome',
 'deriva',
 'latim',
 'alexander',
 'romanização',
 'nome',
 'grego',
 'αλέξανδρος',
 'aléksandros',
 'etimologicamente',
 'nome',
 'composto',
 'verbo',
 'ἀλέξειν',
 'aléksein',
 'defender',
 'substantivo',
 'ἀνδρός',
 'andrós',
 'genitivo',
 'ἀνήρ',
 'anēr',
 'homem',
 'assim',
 'pode',
 'ser',
 'traduzido',
 'protetor',
 'humanidade',
 'carece',
 'fontes',
 'termo',
 'tipo',
 'raro',
 'composto',
 'tatpurusha',
 'invertido',
 'modificante',
 'segunda',
 'posição',
 'tatpurusha',
 'cognato',
 'sânscrito',
 'sendo',
 'nararakṣa',
 'cf',
 'ramayana',
 'equivalente',
 'exato',
 'sânscrito',
 'rakṣinara',
 'pie',
 'composto',
 'gasto',
 'tipo',
 'terpsimbrotos',
 'cujo',
 'significado',
 'original',
 'protege',
 'homens',
 'carece',


In [15]:
from collections import Counter
word_counts = Counter(stopwords(all_words_clean))

In [16]:
word_counts_list = list(word_counts.items())

In [17]:
word_counts_list_sorted = sorted(word_counts_list, key=lambda x: (-x[1], x[0]))

In [18]:
vocab = word_counts_list_sorted[:10000] #10000 mais frequentes

In [19]:
vocab = dict(vocab) #Vocabulario de palavras

In [20]:
vocab

{'categoria': 27940,
 'the': 6359,
 'ser': 4657,
 'ligações': 4248,
 'anos': 4051,
 'of': 3790,
 'brasil': 3613,
 'cidade': 3550,
 'rio': 3272,
 'sobre': 3265,
 'externas': 3057,
 'ano': 3054,
 'estados': 3040,
 'portal': 2968,
 'estado': 2946,
 'onde': 2875,
 'grande': 2862,
 'nbsp': 2855,
 'primeira': 2662,
 'durante': 2643,
 'filmes': 2563,
 'após': 2562,
 'unidos': 2539,
 'primeiro': 2528,
 'história': 2513,
 'parte': 2487,
 'dois': 2470,
 'ver': 2420,
 'nome': 2396,
 'jogos': 2373,
 'futebol': 2356,
 'janeiro': 2319,
 'paulo': 2252,
 'citar': 2185,
 'sendo': 2177,
 'image': 2168,
 'região': 2166,
 'outros': 2164,
 'dia': 2138,
 'grupo': 2127,
 'segundo': 2112,
 'nova': 2098,
 'label': 2094,
 'and': 2057,
 'ainda': 2053,
 'família': 2035,
 'link': 2006,
 'in': 1991,
 'área': 1949,
 'portugal': 1945,
 'sul': 1909,
 'tempo': 1902,
 'três': 1883,
 'série': 1866,
 'pode': 1787,
 'final': 1767,
 'nacional': 1755,
 'referências': 1727,
 'século': 1719,
 'álbum': 1719,
 'contra': 1704,
 '

In [20]:
def candidates(word):
    if word in vocab:
        candidatos = [word]
    else:
        candidatos = []
    candidatos += \
        [w for w in edit1(word) if w in vocab] \
        + [w for w in edit2(word) if w in vocab] \
        + [word] #Perguntar por que precisa

In [21]:
def P(word, N=sum(vocab.values())): return vocab[word] / N

# Stopwords

In [22]:
all_words_per_doc = []
for item in tqdm(data):
    texto = item['body']
    tokens = word_tokenize(texto)
    tokens = limpa_tokens(tokens)
    all_words_per_doc.append(tokens)

100%|█████████████████████████████████████████████████████████████████████████| 11225/11225 [00:00<00:00, 37509.80it/s]


In [23]:
from collections import Counter

all_words_per_doc_single = []
for doc in all_words_per_doc:
    all_words_per_doc_single += set(doc)
doc_counts = Counter(all_words_per_doc_single)

In [24]:
doc_counts_list = list(doc_counts.items())

doc_counts_list_sorted = sorted(doc_counts_list, key=lambda x: (-x[1], x[0]))

In [25]:
doc_counts_list_sorted

[]

In [26]:
from nltk.metrics.distance import edit_distance

In [27]:
LOWERCASE = [chr(x) for x in range(ord('a'), ord('z') + 1)]
LOWERCASE_OTHERS = ['ç', 'á', 'â', ]  # etc.